In [3]:
################################
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
################################
# save data in better form 
#(only once, only relevant data, concat features from logmel and prosody)
import numpy as np
import pickle

##dictionary for translating labels
labelDic = {'angry': 0, 'happy': 1, 'sad': 2, 'neutral': 3}

# Load logmel for training
f = open("data_logmel.train", "rb")
pickle.load(f)  # ignore ids
train_features = pickle.load(f)
train_labels = pickle.load(f)
train_labels = [labelDic[label] for label in train_labels]
f.close()
# load prosody for training
f = open("data_prosody.train", "rb")
pickle.load(f)  # ignore ids
train_features = np.concatenate((train_features, pickle.load(f)), axis=2)
pickle.load(f)  # ignore labels, they are the same
f.close()

# Load logmel for validation
f = open("data_logmel.valid", "rb")
pickle.load(f)  # ignore ids
valid_features = pickle.load(f)
valid_labels = pickle.load(f)
valid_labels = [labelDic[label] for label in valid_labels]
f.close()
# load prosody for validation
f = open("data_prosody.valid", "rb")
pickle.load(f)  # ignore ids
valid_features = np.concatenate((valid_features, pickle.load(f)), axis=2)
pickle.load(f)  # ignore labels, they are the same
f.close()

# Load logmel for testing
f = open("data_logmel.test", "rb")
pickle.load(f)  # ignore ids
test_features = pickle.load(f)
f.close()
# load prosody for testing
f = open("data_prosody.test", "rb")
test_ids = pickle.load(f)  # ignore ids
test_features = np.concatenate((test_features, pickle.load(f)), axis=2)
f.close()

# save in better form
with open("data_project", 'wb') as f_out:
    pickle.dump([train_features, train_labels, valid_features, valid_labels, test_features, test_ids], f_out)



In [0]:
################################
# define dimensions
num_frames = 750
num_feats = 33
num_classes = 4


In [0]:


################################
# Load the data
import pickle 
import numpy as np

loadTrueData = True
if loadTrueData:
    f = open('/content/gdrive/My Drive/DL_project/data_project', "rb")
    [train_features_3dim, train_labels_int, valid_features_3dim, valid_labels_int, test_features_3dim, test_ids] = pickle.load(f,encoding='latin1')
else:
    noSamples = 20
    train_features_3dim = np.ones((noSamples,num_frames,num_feats))
    train_labels_int = np.random.randint(num_classes, size = noSamples)
    valid_features_3dim = np.ones((noSamples,num_frames))
    valid_labels_int = np.random.randint(num_classes, size = noSamples)
    test_features_3dim = np.ones((noSamples,num_frames))




In [18]:
################################
# prepare the data
from keras.utils import to_categorical
import numpy as np
train_labels = to_categorical(train_labels_int)
valid_labels = to_categorical(valid_labels_int)
train_features = train_features_3dim[..., np.newaxis]
valid_features = valid_features_3dim[..., np.newaxis]
test_features = test_features_3dim[..., np.newaxis]
fulltrain_features = np.concatenate((train_features,valid_features),axis = 0)
fulltrain_labels = np.concatenate((train_labels,valid_labels),axis = 0)

print(train_features.shape)
print( train_labels.shape)
print( valid_features.shape)
print( valid_labels.shape)
print( test_features.shape)

print( fulltrain_features.shape)
print( fulltrain_labels.shape)

def accuracy(a,b):
  prod = a*b
  sh = prod.shape
  return sum(sum(prod))/sh[0]



(5531, 750, 33, 1)
(5531, 4)
(3898, 750, 33, 1)
(3898, 4)
(3900, 750, 33, 1)
(9429, 750, 33, 1)
(9429, 4)


In [0]:
################################
# Create a model
from keras.models import Sequential, Model
from keras.layers import Input, Multiply, Dense, Conv2D, Flatten, Dropout, BatchNormalization, Activation, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras import regularizers
from keras.losses import categorical_crossentropy
from sklearn.metrics import confusion_matrix, accuracy_score
import sys


def dl_model(x_train, y_train, x_val, y_val,params,y_val_int):
  

  
  
  inputs = Input(shape=(num_frames,num_feats,1))
  
  #first convolutional layer
  conv1 = Conv2D(params['filters_first_conv'], kernel_size=(12,12), strides=(7,7),activation='relu')(inputs)
  conv1 = MaxPooling2D(pool_size=(int(params['poolingSize']), 1), strides=None, padding='valid', data_format=None)(conv1)
  conv1 = Dropout(params['dropout1'])(conv1)
  conv1_flattened = Flatten()(conv1)

  #model for getting size of last layer
  tempModel = Model(input=[inputs], output=conv1_flattened)
  shapeLastLayer = tempModel.layers[-1].output_shape
  
  #attention layer
  #ks_2 = int(params['conv2_kernelSize'])
  #stride_2 = int(0.6*ks_2)
  #attention_conv2 = Conv2D(params['filters_second_conv'], kernel_size=(ks_2,1), strides=(stride_2,1),activation='relu')(conv1)
  #attention_conv2 = Flatten()(attention_conv2)
  #attention_probs = Dense(shapeLastLayer[1], activation='softmax', name='attention_vec')(attention_conv2)
  
  #Merge layers
  #attention_mul = Multiply()([conv1_flattened, attention_probs])
  
  #hidden layer
  dense1 = Dense(params['noNeuronsDense'])(conv1_flattened)
  #dense1 = Dense(params['noNeuronsDense'])(attention_mul)
  dense1 = Activation('relu')(dense1)
  dense1 = BatchNormalization()(dense1)
  dense1 = Dropout(params['dropout3'])(dense1)
  
  #output layer
  output = Dense(num_classes, activation='softmax')(dense1)

  
  #Model
  model = Model(input=[inputs], output=output)
  #model.summary()
  #sgd = optimizers.SGD(params['learningRate'], decay=1e-6, nesterov=True)
  model.compile(optimizer=params['optimizer'],
                  loss=categorical_crossentropy,
                  metrics=['acc'])

  #Class weights to weigh up for imbalanced validation set
  class_weight = {0: 0,
                  1: params['classWeight1'],
                  2: 0,
                  3: 1}
  
  if ITERATION==2:
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
  
  
  history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=int(params['epochs']),
                        verbose=0,
                        class_weight = class_weight)
  
  
  return model


In [30]:
#####################################
#Do hyperparameter optimization
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.optimizers import Adadelta, Adam, rmsprop
from keras.activations import softmax
from keras.losses import categorical_crossentropy, logcosh
from keras.utils.vis_utils import plot_model
from timeit import default_timer as timer
import csv
import pandas as pd

# Global variables
global  ITERATION
ITERATION = 0
global out_file
out_file = "results.csv"
# File to save first results
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)
# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

#Define Function needed for optimizer
def optimize_model(x_train, y_train, x_val, y_val, params,y_val_int):
  # Keep track of evals
  global ITERATION
  ITERATION += 1
  start = timer()
  
  trainedModel = dl_model(x_train, y_train, x_val, y_val,params,y_val_int)
  probs = trainedModel.predict(x_val, batch_size = 512, verbose = 0)
  y_pred = np.argmax(probs,axis=1) 
  acc = accuracy_score(y_val_int, y_pred)
  print('\naccuracy:', acc, ', iter: ', ITERATION, ', confusion matrix:')  
  conf_matrix = confusion_matrix(y_pred,y_val_int)
  print(conf_matrix)
  sys.stdout.flush() 
  
  run_time = timer() - start
  
  # Write to the csv file ('a' means append)

  of_connection = open(out_file, 'a')
  writer = csv.writer(of_connection)
  writer.writerow([acc, params, ITERATION, run_time])
  
  
  return {'loss': -acc, 'status': STATUS_OK}

space = {
    'epochs': hp.quniform('epochs', 10,15,1),
    'poolingSize': hp.quniform('poolingSize', 10,15,1),
    
    #'learningRate': hp.uniform('learningRate',0.03,0.03),
    #'momentum': hp.uniform('momentum',0.7,0.95),
    'optimizer': hp.choice('optimizer',['adam']),
    'batch_size': hp.choice('batch_size', [512]),
    'dropout1': hp.uniform('dropout1', 0.1,0.3),
    #'dropout2': hp.uniform('dropout2', 0.0,0.4),
    'dropout3': hp.uniform('dropout3', 0.1,0.3),
    'noNeuronsDense': hp.choice('noNeuronsDense', [24]),
    #'classWeight0': hp.uniform('classWeight0', 0.0,0.02),
    'classWeight1': hp.uniform('classWeight1', 0.8,1.2),
    #'classWeight2': hp.uniform('classWeight2', 0.0,0.02),
    'weight_regulizer':hp.uniform('weight_regularizer', 0.1,1),
    'filters_first_conv':hp.choice('filters_first_conv', [48]),   
    #'filters_second_conv':hp.choice('filters_second_conv', [32, 48]),
    #'conv2_kernelSize': hp.quniform('conv2_kernelSize',2,5,1)
    #'loss': hp.choice('loss', [categorical_crossentropy]),
        }
  
objective = lambda params: optimize_model(train_features,train_labels,valid_features,valid_labels,params,valid_labels_int) 
trials = Trials()
bestParamsRaw = fmin(objective, space, algo=tpe.suggest, max_evals=30, trials=trials)

print('FINISHED')

  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("fl...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`



accuracy: 0.4976911236531555 , iter:  1 , confusion matrix:
[[   7    9    5    3]
 [ 247  577   98  381]
 [   1    6    7    5]
 [ 141  730  332 1349]]

accuracy: 0.4787070292457671 , iter:  2 , confusion matrix:
[[   0    0    0    1]
 [ 285  772  179  641]
 [   1    2    0    2]
 [ 110  548  263 1094]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("fl...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`



accuracy: 0.4843509492047204 , iter:  3 , confusion matrix:
[[   1    2    1    1]
 [ 133  337   47  183]
 [   1    2    0    4]
 [ 261  981  394 1550]]

accuracy: 0.482811698306824 , iter:  4 , confusion matrix:
[[   2    5    3    8]
 [ 253  667  133  517]
 [   0    0    0    0]
 [ 141  650  306 1213]]

accuracy: 0.4820420728578758 , iter:  5 , confusion matrix:
[[   3    6    1    0]
 [ 186  433   56  289]
 [   5   13    5   11]
 [ 202  870  380 1438]]

accuracy: 0.4856336582863007 , iter:  6 , confusion matrix:
[[   2    1    1    3]
 [ 232  565  101  404]
 [   0    0    0    5]
 [ 162  756  340 1326]]

accuracy: 0.4835813237557722 , iter:  7 , confusion matrix:
[[   0    0    0    0]
 [ 259  686  149  539]
 [   2   10    2    2]
 [ 135  626  291 1197]]

accuracy: 0.4943560800410467 , iter:  8 , confusion matrix:
[[   0    4    2    1]
 [ 266  708  139  511]
 [   2    1    3   10]
 [ 128  609  298 1216]]

accuracy: 0.478450487429451 , iter:  9 , confusion matrix:
[[   0    2    1 

In [31]:
###########################
# show trials with performance and params in sorted table

import ast

# Sort the trials with lowest loss (highest AUC) first
trials_results = sorted(trials.results, key = lambda x: x['loss'])

#read file
results = pd.read_csv(out_file)
# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = False, inplace = True)
results.reset_index(inplace = True, drop = True)

# Create a new dataframe for storing parameters
bayes_params = pd.DataFrame(columns = list(ast.literal_eval(results.loc[0, 'params']).keys()),
                            index = list(range(len(results))))
# Add the results with each parameter a different column
for i, params in enumerate(results['params']):
    bayes_params.loc[i, :] = list(ast.literal_eval(params).values())   
bayes_params['acc'] = results['loss']
bayes_params['iteration'] = results['iteration']
print(bayes_params.head(10))

#get best hyperparameters
from hyperopt import space_eval
bestParams = space_eval(space, trials.argmin)

  batch_size classWeight1  dropout1  dropout3 epochs filters_first_conv  \
0        512       1.0598  0.163829  0.296926     11                 48   
1        512     0.816718  0.149886  0.183898     11                 48   
2        512      1.00469  0.100638  0.121903     14                 48   
3        512     0.819322  0.233692  0.250844     12                 48   
4        512      1.06541  0.279584  0.231624     11                 48   
5        512       1.0748  0.235999  0.169461     15                 48   
6        512      1.19669  0.147109  0.101973     10                 48   
7        512      1.13239  0.205359  0.167776     13                 48   
8        512     0.927701   0.25346  0.222832     10                 48   
9        512      1.05558  0.118918  0.291989     10                 48   

  noNeuronsDense optimizer poolingSize weight_regulizer       acc  iteration  
0             24      adam          11         0.147189  0.497691          1  
1             24

In [32]:
#do final training on best hyperparams and define prediction function
trainedModel = dl_model(fulltrain_features,fulltrain_labels,valid_features,valid_labels,bestParams,valid_labels_int)
predict = lambda x: np.argmax(trainedModel.predict(x, batch_size = 512, verbose = 0),axis=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("fl...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [33]:
################################
# compute confusion matrices

from sklearn.metrics import confusion_matrix, accuracy_score
np.set_printoptions(precision=3, suppress=True)


#Confusion matrix and class distribution for train data
train_classes = predict(train_features)
train_matrix = confusion_matrix(train_classes,train_labels_int)
print("\nConfusion matrix for classifying train data:")
print(train_matrix)
print("\nDistribution of true labels for train data:")
train_dist = np.sum(train_matrix,axis=0)
print(train_dist/sum(train_dist))
print("\nDistribution of predicted labels for train data:")
train_dist_pred = np.sum(train_matrix,axis=1)
print(train_dist_pred/sum(train_dist_pred))
print("\nAccuracy on train data:")
print(accuracy_score(train_classes,train_labels_int))


#Confusion matrix for validation data
valid_classes = predict(valid_features)
valid_matrix = confusion_matrix(valid_classes,valid_labels_int)
print("\n\nConfusion matrix for classifying validation data:")
print(valid_matrix)
print("\nDistribution of true labels for validation data:")
valid_dist = np.sum(valid_matrix,axis=0)
print(valid_dist/sum(valid_dist))
print("\nDistribution of predicted labels for validation data:")
valid_dist_pred = np.sum(valid_matrix,axis=1)
print(valid_dist_pred/sum(valid_dist_pred))
print("\nAccuracy on validation data:")
print(accuracy_score(valid_classes,valid_labels_int))

# Distribution of classified labels for test data
test_classes = predict(test_features)
print("\n\nDistribution of predicted labels for test data:")
test_dist = [sum(test_classes == label) for label in range(4)]
print(test_dist/sum(test_dist))


Confusion matrix for classifying train data:
[[   1    0    3    0]
 [ 654  793  103  137]
 [  17    0    0    0]
 [ 431  843  978 1571]]

Distribution of true labels for train data:
[0.199 0.296 0.196 0.309]

Distribution of predicted labels for train data:
[0.001 0.305 0.003 0.691]

Accuracy on train data:
0.4275899475682517


Confusion matrix for classifying validation data:
[[   0    0    2    0]
 [ 173  515   63  174]
 [   0    0    0    0]
 [ 223  807  377 1564]]

Distribution of true labels for validation data:
[0.102 0.339 0.113 0.446]

Distribution of predicted labels for validation data:
[0.001 0.237 0.    0.762]

Accuracy on validation data:
0.5333504361210878


Distribution of predicted labels for test data:
[0.    0.288 0.    0.712]


In [0]:
################################
#Save labels in file
translateLabels = ['angry','happy','sad','neutral']

with open("3324181_ebner_topic2_result.txt",'w') as wFile:
  for i in range(len(test_ids)):
    strToWrite = test_ids[i].decode('UTF-8') + " " + translateLabels[test_classes[i]] + "\n"
    wFile.write(strToWrite)

  